# MapD Charting Example with Altair

Let's see if we can replicate [this](https://omnisci.github.io/mapd-charting/example/example1.html) MapD charting example in Python with Altair, Vega Lite, and Vega:

![](https://cloud.githubusercontent.com/assets/2932405/25641647/1acce1f2-2f4a-11e7-87d4-a4e80cb262f5.gif)

In [5]:
import altair as alt
import jupyterlab_omnisci.vega_ibis
import ibis

alt.data_transformers.enable('ibis')
alt.renderers.enable('ibis')

conn = ibis.mapd.connect(
    host='metis.mapd.com', user='mapd', password='HyperInteractive',
    port=443, database='mapd', protocol= 'https'
)
t = conn.table("flights_donotmodify")

In [6]:
jupyterlab_omnisci.vega_ibis.display_queries()

SELECT "carrier_name", avg("depdelay") AS depdelay,
       avg("arrdelay") AS arrdelay
FROM (
  SELECT "depdelay", "arrdelay", "carrier_name", "origin_state",
         "dep_timestamp"
  FROM flights_donotmodify
  WHERE ("dep_timestamp" >= '2008-04-08 05:41:08') AND
        ("dep_timestamp" <= '2008-06-23 17:06:51')
) t0
WHERE ((NOT ("depdelay" IS NULL) AND NOT ("depdelay" IS NULL)) AND NOT ("arrdelay" IS NULL)) AND NOT ("arrdelay" IS NULL)
GROUP BY carrier_name

SELECT t0.*
FROM (
  SELECT "origin_state", count(*) AS __count
  FROM (
    SELECT "origin_state", "carrier_name", "dep_timestamp"
    FROM flights_donotmodify
    WHERE ("dep_timestamp" >= '2008-04-08 05:41:08') AND
          ("dep_timestamp" <= '2008-06-23 17:06:51')
  ) t2
  GROUP BY origin_state
) t0
WHERE NOT (t0."__count" IS NULL)

SELECT "origin_state", count(*) AS __count
FROM (
  SELECT "origin_state", "carrier_name", "dep_timestamp"
  FROM flights_donotmodify
  WHERE ("dep_timestamp" >= '2008-04-08 05:41:08') AND
        ("dep_timestamp" <= '2008-06-23 17:06:51')
) t0
GROUP BY origin_state

SELECT "carrier_name", avg("depdelay") AS depdelay,
       avg("arrdelay") AS arrdelay
FROM (
  SELECT "depdelay", "arrdelay", "carrier_name", "origin_state",
         "dep_timestamp"
  FROM flights_donotmodify
) t0
WHERE ((NOT ("depdelay" IS NULL) AND NOT ("depdelay" IS NULL)) AND NOT ("arrdelay" IS NULL)) AND NOT ("arrdelay" IS NULL)
GROUP BY carrier_name

SELECT t0.*
FROM (
  SELECT "origin_state", count(*) AS __count
  FROM (
    SELECT "origin_state", "carrier_name", "dep_timestamp"
    FROM flights_donotmodify
  ) t2
  GROUP BY origin_state
) t0
WHERE NOT (t0."__count" IS NULL)

SELECT "origin_state", count(*) AS __count
FROM (
  SELECT "origin_state", "carrier_name", "dep_timestamp"
  FROM flights_donotmodify
) t0
GROUP BY origin_state

SELECT "carrier_name", avg("depdelay") AS depdelay,
       avg("arrdelay") AS arrdelay
FROM (
  SELECT "depdelay", "arrdelay", "carrier_name", "origin_state",
         "dep_timestamp"
  FROM flights_donotmodify
  WHERE ("dep_timestamp" >= '2008-05-22 09:13:42') AND
        ("dep_timestamp" <= '2008-05-26 21:41:08')
) t0
WHERE ((NOT ("depdelay" IS NULL) AND NOT ("depdelay" IS NULL)) AND NOT ("arrdelay" IS NULL)) AND NOT ("arrdelay" IS NULL)
GROUP BY carrier_name

SELECT t0.*
FROM (
  SELECT "origin_state", count(*) AS __count
  FROM (
    SELECT "origin_state", "carrier_name", "dep_timestamp"
    FROM flights_donotmodify
    WHERE ("dep_timestamp" >= '2008-05-22 09:13:42') AND
          ("dep_timestamp" <= '2008-05-26 21:41:08')
  ) t2
  GROUP BY origin_state
) t0
WHERE NOT (t0."__count" IS NULL)

SELECT "origin_state", count(*) AS __count
FROM (
  SELECT "origin_state", "carrier_name", "dep_timestamp"
  FROM flights_donotmodify
  WHERE ("dep_timestamp" >= '2008-05-22 09:13:42') AND
        ("dep_timestamp" <= '2008-05-26 21:41:08')
) t0
GROUP BY origin_state

SELECT "yearmonthdate_dep_timestamp", count(*) AS __count
FROM (
  SELECT "dep_timestamp", "depdelay", "origin_state", "carrier_name",
         DATE_TRUNC(DAY, "dep_timestamp") AS yearmonthdate_dep_timestamp
  FROM flights_donotmodify
  WHERE NOT ("dep_timestamp" IS NULL)
) t0
GROUP BY yearmonthdate_dep_timestamp

SELECT "carrier_name", avg("depdelay") AS depdelay,
       avg("arrdelay") AS arrdelay
FROM (
  SELECT "depdelay", "arrdelay", "carrier_name", "origin_state",
         "dep_timestamp"
  FROM flights_donotmodify
) t0
WHERE ((NOT ("depdelay" IS NULL) AND NOT ("depdelay" IS NULL)) AND NOT ("arrdelay" IS NULL)) AND NOT ("arrdelay" IS NULL)
GROUP BY carrier_name

SELECT t0.*
FROM (
  SELECT "origin_state", count(*) AS __count
  FROM (
    SELECT "origin_state", "carrier_name", "dep_timestamp"
    FROM flights_donotmodify
  ) t2
  GROUP BY origin_state
) t0
WHERE NOT (t0."__count" IS NULL)

SELECT "origin_state", count(*) AS __count
FROM (
  SELECT "origin_state", "carrier_name", "dep_timestamp"
  FROM flights_donotmodify
) t0
GROUP BY origin_state

SELECT "carrier_name", a

In [7]:
states = alt.selection_multi(fields=['origin_state'])
airlines = alt.selection_multi(fields=['carrier_name'])
dates = alt.selection_interval(fields=['dep_timestamp'], encodings=['x'])

HEIGHT = 800
WIDTH = 1000

flights_by_state = alt.Chart(
    t[t.origin_state, t.carrier_name, t.dep_timestamp],
    title="Total Number of Flights by State"
).transform_filter(
    airlines
).transform_filter(
    dates
).mark_bar().encode(
    x='count()',
    y=alt.Y('origin_state', sort=alt.Sort(encoding='x', order='descending')),
    color=alt.condition(states, alt.ColorValue("steelblue"), alt.ColorValue("grey"))
).add_selection(
    states
).properties(
    height= 2 * HEIGHT / 3,
    width=WIDTH / 2
)


carrier_delay = alt.Chart(
    t[t.depdelay, t.arrdelay, t.carrier_name, t.origin_state, t.dep_timestamp],
    title="Carrier Departure Delay by Arrival Delay (Minutes)"
).transform_filter(
    states
).transform_filter(
    dates
).transform_aggregate(
    depdelay='mean(depdelay)',
    arrdelay='mean(arrdelay)',
    groupby=["carrier_name"]
).mark_point(filled=True, size=200).encode(
    x='depdelay',
    y='arrdelay',
    color=alt.condition(airlines, alt.ColorValue("steelblue"), alt.ColorValue("grey")),
    tooltip=['carrier_name', 'depdelay', 'arrdelay']
).add_selection(
    airlines
).properties(
    height=2 * HEIGHT / 3,
    width=WIDTH / 2
)

time = alt.Chart(
    t[t.dep_timestamp, t.depdelay, t.origin_state, t.carrier_name]
).transform_filter(
    'datum.dep_timestamp != null'
).transform_filter(
    airlines
).transform_filter(
    states
).mark_line().encode(
    alt.X(
        'yearmonthdate(dep_timestamp):T',
    ),
    alt.Y(
        'count():Q',
        scale=alt.Scale(zero=False)
    )
).add_selection(
    dates
).properties(
    height=HEIGHT / 3,
    width=WIDTH + 50
)

# jupyterlab_omnisci.vega_ibis.FALLBACK = False

(flights_by_state | carrier_delay) & time

VConcatChart({
  vconcat: [HConcatChart({
    hconcat: [Chart({
      data: Empty DataFrame
      Columns: [origin_state, carrier_name, dep_timestamp]
      Index: [],
      encoding: FacetedEncoding({
        color: ColorValue({
          condition: ColorValue({
            selection: 'selector007',
            value: 'steelblue'
          }),
          value: 'grey'
        }),
        x: X({
          aggregate: 'count',
          type: 'quantitative'
        }),
        y: Y({
          field: 'origin_state',
          sort: Sort({
            encoding: 'x',
            order: 'descending'
          }),
          type: 'nominal'
        })
      }),
      height: 533.3333333333334,
      mark: 'bar',
      selection: {'selector007': SelectionDef({
        fields: ['origin_state'],
        type: 'multi'
      })},
      title: 'Total Number of Flights by State',
      transform: [FilterTransform({
        filter: {'selection': 'selector008'}
      }), FilterTransform({
        filte